### Importing all the required objects and assigning the URL from which the data needs to extracted

In [ ]:
URL = "https://en.wikipedia.org/wiki/Taoiseach"
from gazpacho import Soup,get
import pandas as pd
html = get(URL)

soup = Soup(html)

### Using Pandas to read the required table

In [ ]:
df = pd.read_html(str(soup.find('table')))[3]
df = df.loc[1:,2:]
df


### Assigning new Column labels to the above prepared Dataframe

In [ ]:
df.columns=['Name','TermofOfficeStartDate','TermofOfficeEndDate','Party','Tenure','Exec.CouCompParty','vp','VicePresident','Dale']

In [ ]:
df

### Extracting all the required columns and rows from the Dataframe using the index numbers

In [ ]:
df = df.loc[2:,]
df = df.drop([10,11],axis = 0)


### Splitting the President name and the Constituency details

In [ ]:
df["President_Name"] = df["Name"].str.split("(",expand = True)[0]

In [ ]:
df.head(5)

In [ ]:
df["Age"] = df["Name"].str.split("(",expand = True)[1].str.split(")",expand= True)[0]

In [ ]:
df["Constituency"] = df["Name"].str.split("(",expand = True)[1].str.split(")",expand= True)[1]

In [ ]:
df.head(5)

In [ ]:
df["Constituency1"] = df["Constituency"].str.split("TD for",expand = True)[1]

In [ ]:
df["Constituency2"] = df["Constituency"].str.split("TD for",expand = True)[2]

In [ ]:
df = df.drop('vp', axis = 1)

In [ ]:
df

In [ ]:
df = df.drop("Constituency",axis="columns")

In [ ]:
import numpy as np

In [ ]:
df.index = np.arange(1,len(df) + 1)

In [ ]:
df

In [ ]:
df
type(df)

### To change the date format for the start and end dates of Office and also adding today's date to the ongoing Tenure

In [ ]:
df['TermofOfficeStartDate'] = df['TermofOfficeStartDate'].str.split('[',expand = True)[0]

df['TermofOfficeEndDate'] = df['TermofOfficeEndDate'].str.split('[',expand = True)[0]




In [ ]:
from datetime import datetime
df = df.replace(df.loc[44]['TermofOfficeEndDate'],datetime.now())


df['TermofOfficeStartDate'] = pd.to_datetime(df['TermofOfficeStartDate'])

In [ ]:
df

In [ ]:
df['TermofOfficeEndDate'] = pd.to_datetime(df['TermofOfficeEndDate'])

df['TermofOfficeEndDate'] = df['TermofOfficeEndDate'].astype(str)

df['TermofOfficeEndDate'] = df['TermofOfficeEndDate'].str.split(' ', expand = True)[0]

df['TermofOfficeEndDate'] 

df['TermofOfficeEndDate'] = pd.to_datetime(df['TermofOfficeEndDate'])

In [ ]:
df

### SQL Statements to create the Database and the table

In [ ]:
config = {
    'user' : 'electionuser',
    'password' : 'electionp@wd',
    'host' : '127.0.0.1',
    'database' : 'ElectionDB'
}

In [ ]:
import DBcm

In [ ]:
with DBcm.UseDatabase(config) as Cursor:
    SQL = """INSERT INTO President(Name,TOO_StrtDate,TOO_EndDate,Party,Tenure,EC_CompParty,VicePresident,Dale,PresidentName,Age,Constituency1,Constituency2) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    i=0
    while i < len(df):
        data = [row for row in df.iloc[i]]
        data1 = [tuple(data)]
        Cursor.executemany(SQL,data1)
        i+=1
        

### (a) List of constituencies in Ireland which have produced a Taoiseach - Answer

### (b) Political party has produced the most individual Taoiseachs

### Creating a new dataframe 

In [ ]:
vpdf = df[["TermofOfficeEndDate","VicePresident","President_Name"]]

In [ ]:
vpdf

### (c) List of Presidents who previously held the post of Vice President 

In [ ]:
data = []
i=0
j=0
bflag = False
for i in range(0,len(vpdf)-1):
    for j in range(0,len(vpdf)-2):
        if i< len(vpdf) and j< len(vpdf):
            if vpdf.iloc[i]["VicePresident"] == vpdf.iloc[j+1]["President_Name"]:
                if data != vpdf.iloc[i]["VicePresident"]:
                    data.append(vpdf.iloc[i]["VicePresident"])
                    bflag = True
                    if bflag == True:
                        break

In [ ]:
data

In [ ]:
data = set(data)

### Answer to the above question (Question - c)

In [ ]:
data

In [ ]:
df

### Creating a new dataframe

In [ ]:
vpdf12 = df[["President_Name","VicePresident","Party","TermofOfficeStartDate","TermofOfficeEndDate"]]

In [ ]:
vpdf12

### To calculate Years,Months and Days of each office

In [ ]:
val = ((vpdf12['TermofOfficeEndDate'] - vpdf12['TermofOfficeStartDate'])/np.timedelta64(1,'Y'))

In [ ]:
val12 = val.astype(str)

In [ ]:
val

In [ ]:
vpdf12['Years'] = val12.str.split('.',expand = True)[0]

In [ ]:
x = val12.str.split('.',expand = True)[1]

In [ ]:
x2 = "0." +x


In [ ]:
x3 = x2.astype(float)

In [ ]:
month_val = x3*12


In [ ]:
month_val

In [ ]:
month_val = month_val.astype(str)

In [ ]:
upd_month_val = month_val.str.split('.',expand = True)

In [ ]:
vpdf12['Months'] = upd_month_val[0]

In [ ]:
vpdf12

In [ ]:
upd_month_val[1]

In [ ]:
days = upd_month_val[1]

In [ ]:
days_new = "0." + days

In [ ]:
days_update = days_new.astype(float)

In [ ]:
days_final = days_update *30

In [ ]:
days_final

In [ ]:
days_strval = days_final.astype(str)

In [ ]:
days_splitted = days_strval.str.split('.',expand = True)

In [ ]:
day_1 = days_splitted[0]
day_1 = day_1.astype(float)
day_2 = days_splitted[1]

In [ ]:
day_3 = "0." +day_2

In [ ]:
day_3 = day_3.astype(float)

### We are rounding off all the days which are having decimals greater than 0.2 to the next integer

In [ ]:
for i in range(1,46):
    if day_3[i] >= 0.2:
        day_1[i] = day_1[i] + 1
    else:
        day_1[i] = day_1[i]

In [ ]:
day_1

In [ ]:
vpdf12['Days'] = day_1

In [ ]:
vpdf12

### Creating a new table for further analysis

In [ ]:
with DBcm.UseDatabase(config) as Cursor:
    SQL = """INSERT INTO PresidentTerm(PresidentName,VicePresident,Party,TOO_StrtDate,TOO_EndDate,Years,Months,Days) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
    i=0
    while i < len(vpdf12):
        data = [row for row in vpdf12.iloc[i]]
        data1 = [tuple(data)]
        Cursor.executemany(SQL,data1)
        i+=1
        

In [ ]:
df

In [ ]:
vpdf12

### (e) Taoiseach who held the office for the longest amount of uninterrupted time - Answer

In [ ]:
for i in range(0,44):
    if vpdf12.iloc[i]['President_Name'] == vpdf12.iloc[i+1]['President_Name'] and vpdf12.iloc[i]['TermofOfficeEndDate'] ==  vpdf12.iloc[i+1]['TermofOfficeStartDate']:
        
        var_PresidentName = (vpdf12.iloc[i]['President_Name'])
        var_years = (int(vpdf12.iloc[i]['Years'])+ int(vpdf12.iloc[i+1]['Years']))
        var_months = (int(vpdf12.iloc[i]['Months'])+ int(vpdf12.iloc[i+1]['Months']))
        var_days = (int(vpdf12.iloc[i]['Days'])+ int(vpdf12.iloc[i+1]['Days']))
        print(var_PresidentName,'-' ,var_years,'Years' ,var_months,'Months' ,var_days,'Days')
        
        

### (d) Political party who held the office of Taoiseach for the longest amount

### (f) Individual politician who held the office of Taoiseach for the shortest amount of time

### (g) Political party who held the office of Taoiseach for the largest number of Dáils

### (h) Politician who held the office of Taoiseach for the most number of Dáils

### (i) Age was each Taoiseach on the day they assumed office

### (j) Politician who was the oldest on the last day of their term

###  Which Irish political party is the most successful? 